In [5]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [6]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(i))
    ides.append(preprocessed_ide(i))

TypeError: preprocessed_dialogue() missing 1 required positional argument: 'idx'

In [7]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0

# Max consecutive dialogue by one speaker
for i in dialogue_input:
    maxLength = max(maxLength, len(i))

In [8]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# padding dialogues up to max consecutive dialogue length
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
from sklearn.preprocessing import MinMaxScaler
# scaler = MinMaxScaler()
# print(sample[3])
# sample = scaler.fit_transform(sample)
# print(sample[3])

one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)


for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)
    
print(len(one_hot_encoded_dialogue_input), len(one_hot_encoded_dialogue_output))

ValueError: zero-size array to reduction operation maximum which has no identity

In [9]:
from sklearn.metrics import confusion_matrix
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.LSTM(units=128, input_shape=(21, 43)),
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
    return model

def train_and_evaluate_model(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=2)[1]
        print(score)
        scores.append(score)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = np.array(dialogue_output)

model = build_model()
# model.summary()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)


ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: n_samples=0.